Notebook and solution based on: 
* [https://www.kaggle.com/code/guilhemlemoigne/gan-lsun](https://www.kaggle.com/code/guilhemlemoigne/gan-lsun)
* [https://github.com/Zeleni9/pytorch-wgan](https://github.com/Zeleni9/pytorch-wgan)
* https://github.com/Lornatang/WassersteinGAN_GP-PyTorch/tree/master


# GAN on lsun dataset

This notebook trains a [DCGAN](https://arxiv.org/pdf/1511.06434.pdf) on the lsun dataset.

The code is adapted from [pytorch's DCGAN tutorial](https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html) by Nathan Inkawhich.

I changed the dataset and added image display during training. The results are similar to those in the [article](https://arxiv.org/pdf/1511.06434.pdf). we can see rooms generated after five epochs.

In [ ]:
!pip install neptune

!pip install clean-fid

## Download precomputed FID for LSUN

In [ ]:
import numpy as np
import neptune
from shutil import copyfile
import os
import cleanfid
from cleanfid import fid

In [ ]:
PROJECT = "mstaczek/deep-learning-project-3-gan"
TOKEN = ""

In [ ]:
custom_name = 'lsun_bedroom'

run = neptune.init_run(with_id='DLGAN-18', mode="read-only",    
    project=PROJECT,
    api_token=TOKEN
)

fid_filename = "lsun_bedroom_clean_custom_na.npz"

run[fid_filename].download()

run.stop()

os.makedirs(os.path.join(os.path.dirname(cleanfid.__file__), 'stats'), exist_ok=True)
copyfile(fid_filename + '.npz', os.path.join(os.path.dirname(fid.__file__), 'stats', fid_filename))

custom_name = "lsun_bedroom"
does_fid_exists_now = fid.test_stats_exists(custom_name, mode="clean")
print(f'Successfully loaded FID stats: {does_fid_exists_now}')

### FID with random noise

In [ ]:
#import torch

#fid.compute_fid(
#    gen=lambda x: torch.rand(x.shape[0], 3, 64, 64) * 255, dataset_name=custom_name, dataset_split="custom", 
#    z_dim=100, num_gen=1024, num_workers=2
#)

## Setting parameters

In [ ]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from cleanfid import fid

In [ ]:
# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [ ]:
# Root directory for dataset
dataroot = "../input/lsun_bedroom/data0/lsun/bedroom"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100
nz_1 = 20
nz_2 = 300

# Size of feature maps in generator
ngf = 64
ngf_1 = 16
ngf_2 = 256

# Size of feature maps in discriminator
ndf = 64
ndf_1 = 16
ndf_2 = 256

# Number of training epochs
num_epochs = 20

# Learning rate for optimizers
lr = 0.0002
lr_2 = 0.00005

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
if torch.cuda.is_available():
    ngpu = torch.cuda.device_count()
else:
    ngpu = 0

# images generated for FID score after every epoch
num_gen = 1000

# Nubmer of loop to train ciritic
n_critics = 5

# Weight Clipping Limit
weight_cliping_limit = 0.01

In [ ]:
print(f"Number of GPUs: {ngpu}")

## Dataset initialisation

In [ ]:
# We can use an image folder dataset the way we have it setup.
# Create the dataset
# images in scale (0, 1)
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor()
                           ]))

#dataset = torch.utils.data.Subset(dataset, list(range(0, 500)))

# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

### Network training

In [ ]:
def generate_images(netG, fixed_noise):
    with torch.no_grad():
        fake = netG(fixed_noise).detach().cpu()
    return vutils.make_grid(fake, padding=2, normalize=True)

def calculate_fid(netG, custom_name, nz, num_gen):
    class Generator_fixed(nn.Module):
        def __init__(self, old_model):
            super(Generator_fixed, self).__init__()
            self.main = old_model

        def forward(self, input):
            not_scaled_result = self.main(input.reshape(-1, nz, 1, 1))
            return not_scaled_result * 255
        
    return fid.compute_fid(
        gen=Generator_fixed(netG), dataset_name=custom_name, dataset_split="custom", 
        z_dim=nz, num_gen=num_gen, num_workers=2
    )

def save_model(netG, netD, model_name, epoch):
    os.makedirs('./checkpoints', exist_ok=True)
    os.makedirs(f'./checkpoints/{model_name}', exist_ok=True)
    torch.save(netG, f'./checkpoints/{model_name}/netG_epoch_{epoch}.pth')
    torch.save(netD, f'./checkpoints/{model_name}/netD_epoch_{epoch}.pth')
    
def print_stats(epoch, num_epochs, i, n, errD, errG, D_x, D_G_z1, D_G_z2, scoreGenerator, scoreDiscriminator, scoreReal):
    print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
          % (epoch, num_epochs, i, n, errD, errG, D_x, D_G_z1, D_G_z2))
    if scoreGenerator is not None and scoreDiscriminator is not None and scoreReal is not None:
        print(f"scoreGenerator: {scoreGenerator:.4f}, scoreDiscriminator: {scoreDiscriminator:.4f}, scoreReal: {scoreReal:.4f}")
    
def calculate_scores(y_hat_fake, y_hat_real):
    return y_hat_fake, 0.5 * y_hat_real + 0.5 * (1 - y_hat_fake), y_hat_real

In [ ]:
import math

def generate_images_from_latent_space(model, fixed_noise_1, fixed_noise_2):
    list_of_latent_vectors = []
    list_of_latent_vectors.append(fixed_noise_1)
    diff = (fixed_noise_2 - fixed_noise_1) / 9
    for i in range(1, 9):
        list_of_latent_vectors.append(fixed_noise_1 + diff * i)
    list_of_latent_vectors.append(fixed_noise_2)
    selected_noises = torch.stack(list_of_latent_vectors)
    
    # plot
    fig, ax = plt.subplots(2, 5, figsize=(20, 8))
    output = model(selected_noises)
    output = output.cpu().detach().numpy()
    output = np.transpose(output, (0, 2, 3, 1))
    for i in range(2):
        for j in range(5):
            ax[i, j].imshow(output[i * 5 + j])
            ax[i, j].axis('off')

    for i in range(2):
        for j in range(5):
            ax[i,j].set_title(f'{i*5+j+1}/10', fontsize=25)

    for axi in ax.flat:
        axi.set_xticks([])
        axi.set_yticks([])

    fig.suptitle('Interpolation between two latent vectors', fontsize=35)
    fig.tight_layout()
    
    return fig

In [ ]:
class Trainer:
    def train(netD, netG, model_name, num_epochs=num_epochs, dataloader=dataloader, lr=lr, beta1=beta1, device=device, nz=nz, image_size=image_size):
        def step_train_discriminator_real(netD, data, real_label, criterion, device):
            ############################
            # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            ###########################
            ## Train with all-real batch
            # Format batch
            real_cpu = data[0].to(device)
            b_size = real_cpu.size(0)
            label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
            # Forward pass real batch through D
            output = netD(real_cpu).view(-1)
            # Calculate loss on all-real batch
            errD_real = criterion(output, label)
            # Calculate gradients for D in backward pass
            #errD_real.backward()
            D_x = output.mean().item()

            return errD_real, D_x

        def step_train_discriminator_fake(netD, b_size, nz, fake_label, criterion, device):
            ## Train with all-fake batch
            # Generate batch of latent vectors
            noise = torch.randn(b_size, nz, device=device)
            # Generate fake image batch with G
            fake = netG(noise)
            label = torch.full((b_size,), fake_label, dtype=torch.float, device=device)
            # Classify all fake batch with D
            output = netD(fake.detach()).view(-1)
            # Calculate D's loss on the all-fake batch
            errD_fake = criterion(output, label)
            # Calculate the gradients for this batch, accumulated (summed) with previous gradients
            #errD_fake.backward()
            D_G_z1 = output.mean().item()

            return errD_fake, D_G_z1, fake

        def step_train_generator(netG, b_size, fake, real_label, criterion, device):
            ############################
            # (2) Update G network: maximize log(D(G(z)))
            ###########################
            label = torch.full((b_size,), real_label, dtype=torch.float, device=device) # fake labels are real for generator cost
            # Since we just updated D, perform another forward pass of all-fake batch through D
            output = netD(fake).view(-1)
            # Calculate G's loss based on this output
            errG = criterion(output, label)
            # Calculate gradients for G
            errG.backward()
            D_G_z2 = output.mean().item()

            return errG, D_G_z2
    
        print(f"Starting training {model_name} model")
        fixed_noise_generator = torch.Generator(device=device)
        fixed_noise_generator.manual_seed(123)
        fixed_noise = torch.randn(64, nz, device=device, generator=fixed_noise_generator)
        fixed_noise_generator.manual_seed(4)
        fixed_noise_latent_1 = torch.randn(nz, 1, 1, device=device, generator=fixed_noise_generator)
        fixed_noise_generator.manual_seed(10)
        fixed_noise_latent_2 = torch.randn(nz, 1, 1, device=device, generator=fixed_noise_generator)
        
        criterion = nn.BCELoss()
        real_label = 1.
        fake_label = 0.

        optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
        optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

        img_list = []
        latent_img_list = []
        G_losses = []
        D_losses = []
        fid_scores_per_epoch = []

        scoreGenerator = []
        scoreReal = []
        scoreDiscriminator = []
        iters = 0
        
        netD.train()
        netG.train()

        for epoch in range(num_epochs):
            for i, data in enumerate(dataloader, 0):                
                # DISCRIMINATOR
                netD.zero_grad()
                b_size = data[0].to(device).size(0)
                errD_real, D_x = step_train_discriminator_real(netD, data, real_label, criterion, device)
                errD_fake, D_G_z1, fake = step_train_discriminator_fake(netD, b_size, nz, fake_label, criterion, device)
                errD = errD_real + errD_fake # Compute error of D as sum over the fake and the real batches and update D
                errD.backward()
                optimizerD.step()

                # GENERATOR
                netG.zero_grad()
                errG, D_G_z2 = step_train_generator(netG, b_size, fake, real_label, criterion, device) 
                optimizerG.step() # Update G

                # SAVINGS         
                scores = calculate_scores(D_G_z1, D_x)
                scoreGenerator.append(scores[0])
                scoreDiscriminator.append(scores[1])
                scoreReal.append(scores[2])

                G_losses.append(errG.item())
                D_losses.append(errD.item())

                # Output training stats
                if i % 50 == 0:
                    print_stats(epoch, num_epochs, i, len(dataloader), errD.item(), errG.item(), 
                                D_x, D_G_z1, D_G_z2, scoreGenerator[-1], scoreDiscriminator[-1], scoreReal[-1])            

                # Check how the generator is doing by saving G's output on fixed_noise
                if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
                    netG.eval()
                    img_list.append(generate_images(netG, fixed_noise))
                    latent_img_list.append(generate_images_from_latent_space(netG, fixed_noise_latent_1, fixed_noise_latent_2))
                    netG.train()

                iters += 1

            fid_scores_per_epoch.append(calculate_fid(netG, custom_name, nz, num_gen))
            save_model(netG, netD, model_name, epoch)

        return img_list, latent_img_list, G_losses, D_losses, fid_scores_per_epoch, scoreGenerator, scoreDiscriminator, scoreReal
        
    def train_wasserstein(netD, netG, model_name, n_generator_wait=n_critics, num_epochs=num_epochs, dataloader=dataloader, lr=lr, device=device, nz=nz, image_size=image_size, weight_cliping_limit=weight_cliping_limit):
        def step_train_discriminator_wasserstein(netG, netD, data, noise, device):
            real_images = data[0].to(device)
            batch_size = real_images.size(0)

            real_output = netD(real_images)
            errD_real = torch.mean(real_output)
            D_x = real_output.mean().item()

            fake_images = netG(noise)
            fake_output = netD(fake_images.detach())
            errD_fake = torch.mean(fake_output)
            D_G_z1 = fake_output.mean().item()

            errD = -errD_real + errD_fake
            errD.backward()

            return D_x, D_G_z1, errD

        def step_train_generator_wasserstein(netG, netD, noise):
            fake_images = netG(noise)
            fake_output = netD(fake_images)
            errG = -torch.mean(fake_output)
            D_G_z2 = fake_output.mean().item()
            errG.backward()

            return errG, D_G_z2
    
        print(f"Starting training {model_name} model")
        fixed_noise_generator = torch.Generator(device=device)
        fixed_noise_generator.manual_seed(123)
        fixed_noise = torch.randn(64, nz, device=device, generator=fixed_noise_generator)
        fixed_noise_generator.manual_seed(4)
        fixed_noise_latent_1 = torch.randn(nz, 1, 1, device=device, generator=fixed_noise_generator)
        fixed_noise_generator.manual_seed(10)
        fixed_noise_latent_2 = torch.randn(nz, 1, 1, device=device, generator=fixed_noise_generator)

        optimizerD = optim.RMSprop(netD.parameters(), lr=lr)
        optimizerG = optim.RMSprop(netG.parameters(), lr=lr)

        img_list = []
        latent_img_list = []
        G_losses = []
        D_losses = []
        fid_scores_per_epoch = []
        
        iters = 0
        
        netD.train()
        netG.train()
        
        errG = None
        D_G_z2 = None

        for epoch in range(num_epochs):
            for i, data in enumerate(dataloader, 0):    
                b_size = data[0].to(device).size(0)
                noise = torch.randn(b_size, nz, device=device)
                
                # DISCRIMINATOR
                netD.zero_grad()
                D_x, D_G_z1, errD = step_train_discriminator_wasserstein(netG, netD, data, noise, device)
                optimizerD.step()
                
                # Clip weights of discriminator
                for p in netD.parameters():
                    p.data.clamp_(-weight_cliping_limit, weight_cliping_limit)

                # GENERATOR
                if i % n_generator_wait == 0:
                    netG.zero_grad()
                    errG, D_G_z2 = step_train_generator_wasserstein(netG, netD, noise) 
                    optimizerG.step() # Update G

                # SAVINGS 
                G_losses.append(errG.item())
                D_losses.append(errD.item())

                # Output training stats
                if i % 50 == 0:
                    print_stats(epoch, num_epochs, i, len(dataloader), errD.item(), errG.item(), 
                                D_x, D_G_z1, D_G_z2, None, None, None)            

                # Check how the generator is doing by saving G's output on fixed_noise
                if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
                    netG.eval()
                    img_list.append(generate_images(netG, fixed_noise))
                    latent_img_list.append(generate_images_from_latent_space(netG, fixed_noise_latent_1, fixed_noise_latent_2))
                    netG.train()

                iters += 1

            fid_scores_per_epoch.append(calculate_fid(netG, custom_name, nz, num_gen))
            save_model(netG, netD, model_name, epoch)

        return img_list, latent_img_list, G_losses, D_losses, fid_scores_per_epoch

### DCGAN

In [ ]:
class DCGAN:
    class Generator(nn.Module):
        def __init__(self, ngpu=ngpu, nz=nz, ngf=ngf):
            super(DCGAN.Generator, self).__init__()
            self.ngpu = ngpu
            self.nz = nz
            self.ngf = ngf
            self.main = nn.Sequential(
                DCGAN.Generator.create_block(nz, ngf * 8, 4, 1, 0),
                DCGAN.Generator.create_block(ngf * 8, ngf * 4),
                DCGAN.Generator.create_block(ngf * 4, ngf * 2),
                DCGAN.Generator.create_block(ngf * 2, ngf),
                nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
                nn.Tanh()
            )

        def forward(self, input):
            input = torch.squeeze(input)
            y = self.main(input.reshape(-1, self.nz, 1, 1))
            y = (y + 1) / 2
            return y
        
        def create_block(in_dim, out_dim, kernel_size=4, stride=2, padding=1):
            return nn.Sequential(
                nn.ConvTranspose2d(in_dim, out_dim, kernel_size=kernel_size, stride=stride, padding=padding, bias=False),
                nn.BatchNorm2d(out_dim),
                nn.ReLU(True)
            )
    
    class Discriminator(nn.Module):
        def __init__(self, ngpu=ngpu, nz=nz, ndf=ndf):
            super(DCGAN.Discriminator, self).__init__()
            self.ngpu = ngpu
            self.nz = nz
            self.ndf = ndf
            self.main = nn.Sequential(
                DCGAN.Discriminator.create_block(nc, ndf),
                DCGAN.Discriminator.create_block(ndf, ndf * 2),
                DCGAN.Discriminator.create_block(ndf * 2, ndf * 4),
                DCGAN.Discriminator.create_block(ndf * 4, ndf * 8),
                nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
                nn.Sigmoid()
            )

        def forward(self, input):
            y = self.main(input)
            return y
        
        def create_block(in_dim, out_dim, kernel_size=4, stride=2, padding=1):
            return nn.Sequential(
                nn.Conv2d(in_dim, out_dim, kernel_size=kernel_size, stride=stride, padding=padding, bias=False),
                nn.BatchNorm2d(out_dim),
                nn.LeakyReLU(0.2, inplace=True)
            )
    
    def weights_init(m):
        classname = m.__class__.__name__
        if classname.find('Conv') != -1:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
        elif classname.find('BatchNorm') != -1:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
            nn.init.constant_(m.bias.data, 0)

### Simple GAN

In [ ]:
class SimpleGAN:
    class Generator(nn.Module):
        def __init__(self, ngpu=ngpu, nz=nz, ngf=ngf):
            super(SimpleGAN.Generator, self).__init__()
            self.ngpu = ngpu
            self.nz = nz
            self.ngf = ngf
            self.main = nn.Sequential(
                SimpleGAN.Generator.create_block(nz, ngf),
                SimpleGAN.Generator.create_block(ngf, ngf * 2),
                SimpleGAN.Generator.create_block(ngf * 2, ngf * 4),
                SimpleGAN.Generator.create_block(ngf * 4, ngf * 6),
                nn.Linear(ngf * 4, ngf * ngf * nc),
                nn.Tanh()
            )
        
        def forward(self, input):
            input = torch.squeeze(input)
            y = self.main(input).view(-1, self.nc, self.self.ngf, self.ngf)
            y = (y + 1) / 2
            return y
        
        def create_block(in_dim, out_dim):
            return nn.Sequential(             
                nn.Linear(in_dim, out_dim),
                nn.BatchNorm1d(out_dim, 0.8),
                nn.LeakyReLU(0.25, inplace=True)
            )
    
    class Discriminator(nn.Module):
        def __init__(self, ngpu=ngpu, nz=nz, ndf=ndf):
            super(SimpleGAN.Discriminator, self).__init__()
            self.ngpu = ngpu
            self.nz = nz
            self.ndf = ndf
            self.main = nn.Sequential(
                SimpleGAN.Discriminator.create_block(ndf * ndf * nc, ndf * 6),
                SimpleGAN.Discriminator.create_block(ndf * 6, ndf * 4),
                SimpleGAN.Discriminator.create_block(ndf * 4, ndf * 2),
                SimpleGAN.Discriminator.create_block(ndf * 2, ndf),           
                nn.Linear(ndf, 1),
                nn.Sigmoid()
            )

        def forward(self, input):
            input = input.view(input.size(0), -1)
            return self.main(input)
        
        def create_block(in_dim, out_dim):
            return nn.Sequential(             
                nn.Linear(in_dim, out_dim),
                nn.BatchNorm1d(out_dim, 0.8),
                nn.LeakyReLU(0.25, inplace=True)
            )

### Saving to Neptune function

In [ ]:
def upload_to_neptune(D_losses, G_losses, fid_scores_per_epoch, scoreGenerator, scoreDiscriminator, scoreReal, model_name, img_list, latent_img_list,
                      num_epochs=num_epochs, batch_size=batch_size, image_size=image_size, nz=nz, ngf=ngf, ndf=ndf, lr=lr):
    run = neptune.init_run(
        project=PROJECT,
        api_token=TOKEN,
    )
    params = {
        "num_epochs": num_epochs,
        "batch_size": batch_size,
        "image_size": image_size,
        "latent_vector_size": nz,
        "generator_feature_maps": ngf, 
        "discriminator_feature_maps": ndf,
        "learning_rate": lr,
        "model_name": model_name
    }
    run["parameters"] = params
    
    for i in range(len(G_losses)):
        run["discriminator_loss"].log(D_losses[i])
        run["generator_loss"].log(G_losses[i])
        
        if scoreGenerator is not None:
            run["score_generator"].log(scoreGenerator[i])
        if scoreDiscriminator is not None:
            run["score_discriminator"].log(scoreDiscriminator[i])
        if scoreReal is not None:
            run["score_real"].log(scoreReal[i])
    
    fig, ax = plt.subplots(figsize=(10,5))
    ax.set_title("Generator and Discriminator Loss During Training")
    ax.plot(G_losses,label="G")
    ax.plot(D_losses,label="D")
    ax.set_xlabel("iterations")
    ax.set_ylabel("Loss")
    plt.legend()
    run["losses"].upload(neptune.types.File.as_image(fig))
    
    for filename in os.listdir(f'./checkpoints/{model_name}'):
        if filename.startswith("netG"):
            run["generator_checkpoints/" + filename].upload(f'./checkpoints/{model_name}' + filename)
        elif filename.startswith("netD"):
            run["discriminator_checkpoints/" + filename].upload(f'./checkpoints/{model_name}' + filename)
            
    # upload FID score
    for i in range(len(fid_scores_per_epoch)):
        run["fid_score"].log(fid_scores_per_epoch[i])
      
    # plot fixed noise changes
    for i in range(len(img_list)):
        fig, ax = plt.subplots(figsize=(10,10))
        ax.imshow(np.transpose(img_list[i], (1,2,0)))
        run["fixed_noise_steps"].append(neptune.types.File.as_image(fig)) 
    
    for i in range(len(latent_img_list)):
        run["latent_space_noise_steps"].append(neptune.types.File.as_image(latent_img_list[i]))
        
    run.stop()

## Running codes

In [ ]:
model_name = 'DCGAN'

netG = DCGAN.Generator().to(device)
netD = DCGAN.Discriminator().to(device)

if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))
    netD = nn.DataParallel(netD, list(range(ngpu)))

netG.apply(DCGAN.weights_init)
netD.apply(DCGAN.weights_init)

img_list, latent_img_list, G_losses, D_losses, fid_scores_per_epoch, scoreGenerator, scoreDiscriminator, scoreReal = Trainer.train(netD, netG, model_name)
upload_to_neptune(D_losses, G_losses, fid_scores_per_epoch, scoreGenerator, scoreDiscriminator, scoreReal, model_name, img_list, latent_img_list)

In [ ]:
del netG, netD
torch.cuda.empty_cache()

In [ ]:
model_name = 'DCGAN'

netG = DCGAN.Generator(nz=nz_1).to(device)
netD = DCGAN.Discriminator(nz=nz_1).to(device)

if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))
    netD = nn.DataParallel(netD, list(range(ngpu)))

netG.apply(DCGAN.weights_init)
netD.apply(DCGAN.weights_init)

img_list, latent_img_list, G_losses, D_losses, fid_scores_per_epoch, scoreGenerator, scoreDiscriminator, scoreReal = Trainer.train(netD, netG, model_name, nz=nz_1)
upload_to_neptune(D_losses, G_losses, fid_scores_per_epoch, scoreGenerator, scoreDiscriminator, scoreReal, model_name, img_list, latent_img_list, nz=nz_1)

In [ ]:
del netG, netD
torch.cuda.empty_cache()

In [ ]:
model_name = 'DCGAN'

netG = DCGAN.Generator(nz=nz_2).to(device)
netD = DCGAN.Discriminator(nz=nz_2).to(device)

if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))
    netD = nn.DataParallel(netD, list(range(ngpu)))

netG.apply(DCGAN.weights_init)
netD.apply(DCGAN.weights_init)

img_list, latent_img_list, G_losses, D_losses, fid_scores_per_epoch, scoreGenerator, scoreDiscriminator, scoreReal = Trainer.train(netD, netG, model_name, nz=nz_2)
upload_to_neptune(D_losses, G_losses, fid_scores_per_epoch, scoreGenerator, scoreDiscriminator, scoreReal, model_name, img_list, latent_img_list, nz=nz_2)

In [ ]:
del netG, netD
torch.cuda.empty_cache()

In [ ]:
model_name = 'DCGAN'

netG = DCGAN.Generator(ngf=ngf_1).to(device)
netD = DCGAN.Discriminator(ndf=ndf_1).to(device)

if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))
    netD = nn.DataParallel(netD, list(range(ngpu)))

netG.apply(DCGAN.weights_init)
netD.apply(DCGAN.weights_init)

img_list, latent_img_list, G_losses, D_losses, fid_scores_per_epoch, scoreGenerator, scoreDiscriminator, scoreReal = Trainer.train(netD, netG, model_name)
upload_to_neptune(D_losses, G_losses, fid_scores_per_epoch, scoreGenerator, scoreDiscriminator, scoreReal, model_name, img_list, latent_img_list, ngf=ngf_1, ndf=ndf_1)

In [ ]:
del netG, netD
torch.cuda.empty_cache()

In [ ]:
model_name = 'DCGAN'

netG = DCGAN.Generator(ngf=ngf_2).to(device)
netD = DCGAN.Discriminator(ndf=ndf_2).to(device)

if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))
    netD = nn.DataParallel(netD, list(range(ngpu)))

netG.apply(DCGAN.weights_init)
netD.apply(DCGAN.weights_init)

img_list, latent_img_list, G_losses, D_losses, fid_scores_per_epoch, scoreGenerator, scoreDiscriminator, scoreReal = Trainer.train(netD, netG, model_name, num_epochs=5)
upload_to_neptune(D_losses, G_losses, fid_scores_per_epoch, scoreGenerator, scoreDiscriminator, scoreReal, model_name, img_list, latent_img_list, ngf=ngf_2, ndf=ndf_2, num_epochs=5)

In [ ]:
del netG, netD
torch.cuda.empty_cache()

In [ ]:
model_name = 'SimpleGAN'

netG = SimpleGAN.Generator().to(device)
netD = SimpleGAN.Discriminator().to(device)

if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))
    netD = nn.DataParallel(netD, list(range(ngpu)))

img_list, latent_img_list, G_losses, D_losses, fid_scores_per_epoch, scoreGenerator, scoreDiscriminator, scoreReal = Trainer.train(netD, netG, model_name)
upload_to_neptune(D_losses, G_losses, fid_scores_per_epoch, scoreGenerator, scoreDiscriminator, scoreReal, model_name, img_list, latent_img_list)

In [ ]:
del netG, netD
torch.cuda.empty_cache()

In [ ]:
model_name = 'DCGAN-Wasserstein'

netG = DCGAN.Generator().to(device)
netD = DCGAN.Discriminator().to(device)

# remove sigmoid
removed = list(netD.main.children())[:-1]
netD.main = torch.nn.Sequential(*removed)

if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))
    netD = nn.DataParallel(netD, list(range(ngpu)))

netG.apply(DCGAN.weights_init)
netD.apply(DCGAN.weights_init)

img_list, latent_img_list, G_losses, D_losses, fid_scores_per_epoch = Trainer.train_wasserstein(netD, netG, model_name)
upload_to_neptune(D_losses, G_losses, fid_scores_per_epoch, None, None, None, model_name, img_list, latent_img_list)

In [ ]:
del netG, netD
torch.cuda.empty_cache()

In [ ]:
model_name = 'DCGAN-Wasserstein'

netG = DCGAN.Generator().to(device)
netD = DCGAN.Discriminator().to(device)

# remove sigmoid
removed = list(netD.main.children())[:-1]
netD.main = torch.nn.Sequential(*removed)

if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))
    netD = nn.DataParallel(netD, list(range(ngpu)))

netG.apply(DCGAN.weights_init)
netD.apply(DCGAN.weights_init)

img_list, latent_img_list, G_losses, D_losses, fid_scores_per_epoch = Trainer.train_wasserstein(netD, netG, model_name, lr=lr_2)
upload_to_neptune(D_losses, G_losses, fid_scores_per_epoch, None, None, None, model_name, img_list, latent_img_list, lr=lr_2)

In [ ]:
del netG, netD
torch.cuda.empty_cache()